In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn

In [2]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(1998,2018)]

In [3]:
# Preparing the monitoring of the loop
start_time = time()
requests = 0

In [5]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 70:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:72; Frequency: 0.07753356436088173 requests/s


In [6]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes})

In [7]:
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Data columns (total 5 columns):
movie        2947 non-null object
year         2947 non-null object
imdb         2947 non-null float64
metascore    2947 non-null int64
votes        2947 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 115.2+ KB
None


,movie,year,imdb,metascore,votes
0,Gladiator,(2000),8.5,67,1149580
1,Memento,(2000),8.5,80,987036
2,Snatch,(2000),8.3,55,690566
3,Requiem for a Dream,(2000),8.3,68,670122
4,X-Men,(2000),7.4,64,514781
5,Cast Away,(2000),7.8,73,452832
6,American Psycho,(2000),7.6,64,411677
7,Unbreakable,(2000),7.3,62,302318
8,Meet the Parents,(2000),7.0,73,285395
9,Mission: Impossible II,(2000),6.1,59,277621


In [8]:
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes']]
movie_ratings.head()

,movie,year,imdb,metascore,votes
0,Gladiator,(2000),8.5,67,1149580
1,Memento,(2000),8.5,80,987036
2,Snatch,(2000),8.3,55,690566
3,Requiem for a Dream,(2000),8.3,68,670122
4,X-Men,(2000),7.4,64,514781


In [9]:
movie_ratings.loc[:,'year']=movie_ratings['year'].str[-5:-1].astype(int)

In [10]:
movie_ratings['year'].head(3)

0    2000
1    2000
2    2000
Name: year, dtype: int32